## Importing Libraries

In [1]:
# Data Processing
import numpy as np
import pandas as pd
from collections import Counter

# Importing tde
import tde

# Visualization
import matplotlib.pyplot as plt

# Model and Performance
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import OneClassSVM
from sklearn.metrics import classification_report #for model performance evaluation

# To generate the for loops
import glob
import os
from matplotlib import ticker

## Reading the data

In [2]:
# Read temperature data
temperature_data = pd.read_excel('/Users/yoddmena/Downloads/Research/Telemetry_edited/Ex 01 - 05_06_19 - Edited.xlsx')

# Read infection time data
infection_data = pd.read_excel('/Users/yoddmena/Downloads/Research/Telemetry_edited/MRSA - Master sheet.xlsx', index_col=0)

In [3]:
# Read temperature data
folder = "/Users/yoddmena/Downloads/Research/Telemetry_edited/"

files = glob.glob( folder + "Ex *.xlsx" ) 

minutes = []
temp = []

k = 0
for file in files:
    k = k +1
    if k > 1:
        break
    df = pd.read_excel(file) # reading the excel file
    
    file_min = df.iloc[:,0] #getting only the minutes
    file_temp = df.iloc[:,3::2] # slicing the data for only temperature
    
    minutes.append(file_min) # adding minutes into the list
    temp.append(file_temp) # adding the file into the list

In [4]:
infection_data.head()
infection_data.shape[1]

10

In [7]:
infection_data_1 = infection_data.iloc[:,[1,9]]
infection_data_1.head()

,Strain-ID,Aprprox Infection Minute
Strain,,
C57Bl6,C57Bl6-100,14200
C57Bl6,C57Bl6-101,14200
CC002,CC002-557,14200
CC002,CC002-558,14200
CC002,CC002-559,14200


In [12]:
# Convert the list to a DataFrame
df_temp = pd.DataFrame(temp)
df_temp.head()

ValueError: Must pass 2-d input. shape=(1, 20192, 11)

In [11]:
transpose_temp = temp.T

AttributeError: 'list' object has no attribute 'T'

In [ ]:
# Extract time series from temperature data
time_series = temperature_data.iloc[:, 3:]  # Assuming the time series is in columns after the third column

# Convert the time series data to the desired format (e.g., numeric)
time_series = time_series.apply(pd.to_numeric, errors='coerce')

In [ ]:
# Create a list of infection times
infection_times = infection_data.iloc[:, [9]].copy()
infection_times.head()

In [ ]:
# Plot the time series
plt.plot(time_series)

# Add a vertical line for the specific infection time
plt.axvline(x=infection_time, color='r', linestyle='--')

# Customize the plot as needed (e.g., add labels, titles, etc.)
plt.xlabel('Minute')
plt.ylabel('Temperature')
plt.title('Temperature Time Series with Infection Times')

# Show the plot
plt.show()

In [ ]:
# Assuming df is your DataFrame with the original column names
df = pd.DataFrame({'A_col1': [1, 2, 3], 'B_col2': [4, 5, 6], 'C_col3': [7, 8, 9]})

# Renaming columns by removing a specific substring
df.columns = df.columns.str.replace('_col', '_')

# Print the modified DataFrame
print(df)